# ATS Weekly Automation ( LFD & HTV & others)

**Purpose**\
    Generate the Monday morning Available-To-Stock (ATS) report for mainly LFD and HTV. Also can be used for Monitor and other ATS files updates.

**What it does**\
    • Inputs from manually-downloaded files (GSCM, SAP, and email attachments).\
    • Cleans/standardizes data and applies formatting.\
    • Aggregates by SKU and account; computes ATS.\
    • Writes a finalized, report-ready Excel output.

**How to run**\
    1) Download latest source files and update required params (see following CONFIG).\
    2) Execute the script; no further interaction needed.\
    Runtime is ~10–15 minutes depending on file sizes.

**Why it matters**\
    Replaces ~2–3 hours of Monday manual work, improves accuracy\
    and frees time for other rush issues 😌

Import packages and all necessary modules

In [3]:
import time
from tarfile import version

import openpyxl
from openpyxl import load_workbook
from openpyxl.descriptors import (String,Sequence,Integer)
from openpyxl.descriptors.serialisable import Serialisable
from openpyxl.styles import numbers,Alignment,PatternFill,Font,colors
from openpyxl.utils import get_column_letter
from openpyxl.styles import NamedStyle

import numpy as np, pandas as pd
from pandas import DataFrame
import copy

from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

import warnings
import win32com.client as win32
import os
from sphinx.cmd.quickstart import nonempty
from sqlalchemy import false

**Manuel Work Here**\
Input time and date for data searching and correct output 

In [ ]:
today=input("what day is today? (yyyy,mm,dd)")
today=datetime.strptime(today,"%Y,%m,%d")
week=today.strftime("%V")
yr=today.strftime("%Y")
wk=yr+week
mon=today.strftime("%b")
next_month=today+relativedelta(months=+1)
next_mon=next_month.strftime("%b")
print('DONE!')

Ignore warning for Python, no impact on performace

In [6]:
warnings.filterwarnings('ignore',category=UserWarning)

Define all functions, including extracting data from GSCM and SAP, cleanning data, formatting, calculation and output\
DO NOT EDIT

In [8]:
def clean(ws,maxcol):
    for col in ws.iter_cols(max_col=maxcol,max_row=len(ws['B'])):
        for cell in col:
            cell.value=None
            cell.font = NamedStyle().font
            cell.fill = NamedStyle().fill
            cell.border = NamedStyle().border
            cell.alignment = NamedStyle().alignment
            cell.number_format = NamedStyle().number_format

In [10]:
# data_col: last data column, might following by Pivot table
# arrive_push: PSI arrive, if push the last week to next month for sales support, default is YES
# piv: 0 means no pivot table in the sheet, 1 means yes and refresh, default Yes
def gscm(target, gfile, tsheet, arrive_push=1, piv=1):
    with pd.ExcelWriter(target, mode='a', engine='openpyxl', if_sheet_exists="overlay") as writer:
        workbook = writer.book
        w1 = writer.sheets[tsheet]
        w2_wb = load_workbook(gfile)
        w2 = w2_wb.active

        found_maxcol = 0
        for cols in w1.iter_cols(min_row=1,max_row=1):
            for cell in cols:
                if cell.value == "Total":
                    maxcol=cell.column
                    found_maxcol = 1
            if found_maxcol == 1:
                break

        clean(w1, maxcol)

        for row in w2.iter_rows(min_row=2, min_col=2):
            for cell in row:
                nc = w1.cell(row=cell.row - 1, column=cell.column - 1, value=cell.value)
                if cell.has_style:
                    nc.font = copy.copy(cell.font)
                    nc.fill = copy.copy(cell.fill)
                    nc.border = copy.copy(cell.border)
                    nc.alignment = copy.copy(cell.alignment)
                    nc.number_format = copy.copy(cell.number_format)

        found_maxrow = 0
        for rows in w1.iter_rows(min_col=5, max_col=5):
            for cell in rows:
                if cell.value is None:
                    maxrow = cell.row
                    found_maxrow = 1
            if found_maxrow == 1:
                break
        if found_maxrow == 0:
            maxrow = cell.row + 1

        mon_idx=0
        nondate_idx=0
        next_mon_idx=0
        wk_idx=0
        if tsheet == "PSI Arrivals":
            for toprow in w1.iter_rows(min_row=1, max_row=1):
                    for header in toprow:
                        if header.value == mon:
                            mon_idx = header.col_idx
                        elif header.value == "Category":
                            nondate_idx = header.col_idx
                        elif header.value == next_mon:
                            next_mon_idx = header.col_idx
                        elif header.value == wk:
                            wk_idx = header.col_idx
                        elif wk_idx != 0 and next_mon_idx != 0 and nondate_idx != 0 and mon_idx != 0:
                            break
            wk_qty = mon_idx - wk_idx
            nxwk_qty = next_mon_idx - mon_idx

            for i in range(3,maxrow):
                j = 0
                k = 1
                cur_total = 0
                nx_total = 0
                if arrive_push != 1:
                    while j < wk_qty:
                        wk_select = wk_idx + j
                        cur_total = cur_total + w1.cell(i, wk_select).value
                        j += 1
                else:
                    while j < wk_qty - 1:
                        wk_select = wk_idx + j
                        cur_total = cur_total + w1.cell(i, wk_select).value
                        j += 1
                    nx_total = nx_total + w1.cell(i, mon_idx - 1).value
                    while k < nxwk_qty:
                        wk_select = mon_idx + k
                        nx_total = nx_total + w1.cell(i, wk_select).value
                        k += 1
                    w1.cell(i, next_mon_idx).value = nx_total
                w1.cell(i, mon_idx).value = cur_total


        elif tsheet=="AP1":
            for toprow in w1.iter_rows(min_row=1, max_row=1):
                    for header in toprow:
                        if header.value == mon:
                            mon_idx = header.col_idx
                        elif header.value == "Category":
                            nondate_idx = header.col_idx
                        elif header.value == wk:
                            wk_idx = header.col_idx
                        elif wk_idx != 0 and nondate_idx != 0 and mon_idx != 0:
                            break
            wk_qty = mon_idx - wk_idx
            for i in range(7,maxrow):
                # 5 units in one loop
                j = 0
                cur_total = 0
                if (i-6) % 5 !=2 :
                    while j < wk_qty:
                        wk_select = wk_idx + j
                        cur_total = cur_total + w1.cell(i, wk_select).value
                        j += 1
                    w1.cell(i, mon_idx).value = cur_total
                else:
                    wk_select = wk_idx + j
                    cur_total = cur_total + w1.cell(i+1, wk_select).value
                    j=1
                    while j < wk_qty:
                        wk_select = wk_idx + j
                        cur_total = cur_total + w1.cell(i, wk_select).value
                        j += 1
                    w1.cell(i, mon_idx).value = cur_total

        if piv == 1:
            pivot = w1._pivots[0]
            pivot.cache.refreshOnLoad = True


In [12]:
# ATS files without first 3 rows !!!

def sap(target, sfile, tsheet, piv=1):
    with pd.ExcelWriter(target, mode='a', engine='openpyxl', if_sheet_exists="overlay") as writer:
        workbook = writer.book
        w1 = writer.sheets[tsheet]
        w2_wb = load_workbook(sfile)
        w2 = w2_wb.active

        if tsheet == "Open SO":
            clean(w1, 54)
            for row in w2.iter_rows(min_row=1, min_col=1):
                for cell in row:
                    nc = w1.cell(row=cell.row, column=cell.column, value=cell.value)
                    if cell.has_style:
                        nc.font = copy.copy(cell.font)
                        nc.fill = copy.copy(cell.fill)
                        nc.border = copy.copy(cell.border)
                        nc.alignment = copy.copy(cell.alignment)
                        nc.number_format = copy.copy(cell.number_format)

            found_maxrow = 0
            for rows in w1.iter_rows(min_col=5, max_col=5):
                for cell in rows:
                    if cell.value is None or cell.value.strip() == '':
                        maxrow = cell.row
                        found_maxrow = 1
                if found_maxrow == 1:
                    break
            if found_maxrow == 0:
                maxrow = cell.row + 1

            nonempty_row=[maxrow]
            for rows in w1.iter_rows(min_row=2,min_col=25,max_col=25):
                cell=rows[0]
                if isinstance(cell.value,str) and cell.value.strip()=='':
                    cell.value=None
                if cell.value is not None :
                    nonempty_row.append(cell.row)

            for row in sorted(nonempty_row,reverse=True):
                w1.delete_rows(row)

        elif tsheet=="On-Hand":
            clean(w1, 31)
            for row in w2.iter_rows(min_row=1, min_col=1):
                for cell in row:
                    nc = w1.cell(row=cell.row, column=cell.column+1, value=cell.value)
                    if cell.has_style:
                        nc.font = copy.copy(cell.font)
                        nc.fill = copy.copy(cell.fill)
                        nc.border = copy.copy(cell.border)
                        nc.alignment = copy.copy(cell.alignment)
                        nc.number_format = copy.copy(cell.number_format)

            found_maxrow = 0
            for rows in w1.iter_rows(min_col=5, max_col=5):
                for cell in rows:
                    if cell.value is None or cell.value.strip() == '':
                        maxrow = cell.row
                        found_maxrow = 1
                if found_maxrow == 1:
                    break
            if found_maxrow==0:
                maxrow = cell.row+1

            w1['A1'].value="OH minus DO"
            for i in range(2,maxrow):
                w1.cell(i,1).value='=J{}-Q{}'.format(i,i)


        if piv == 1:
            pivot = w1._pivots[0]
            pivot.cache.refreshOnLoad = True

In [14]:
def Sum(ats,today):
    mon=today.strftime("%b")
    next_month=today+relativedelta(months=+1)
    next_mon=next_month.strftime("%b")
    two_month=today+relativedelta(months=+2)
    two_mon=two_month.strftime("%b")
    three_month=today+relativedelta(months=+3)
    three_mon=three_month.strftime("%b")
    four_month=today+relativedelta(months=+4)
    four_mon=four_month.strftime("%b")
    
    with pd.ExcelWriter(ats, mode='a', engine='openpyxl', if_sheet_exists="overlay") as writer:
        workbook = writer.book
        ws = writer.sheets['Summary']
        ws.delete_cols(16,15)
        del_row=[]
        for row in range(2,ws.max_row + 1 ):
            if ws.cell(row,4).value == "OUTDOOR LCD":
                del_row.append(row)
                continue
            
            m1=int(ws.cell(row,8).value)
            m2=int(ws.cell(row,9).value)
            m3=int(ws.cell(row,10).value)
            m4=int(ws.cell(row,11).value)
            m5=int(ws.cell(row,12).value)
            
            # need Esther help algorithm
            
            if (m1+m2+m3)<0:
                ws.cell(row,8).value = 0
                ws.cell(row,9).value = 0
                ws.cell(row,10).value = 0
                
            elif m1>=0 and m2 >=0 and m3<0 and (m2+m3)<0:
                ws.cell(row,8).value = (m1+m2+m3)
                ws.cell(row,9).value = 0
                ws.cell(row,10).value = 0
            elif m1>=0 and m2 >=0 and m3<0 and (m2+m3)>=0:
                ws.cell(row,9).value = (m2+m3)
                ws.cell(row,10).value = 0
            elif m1>=0 and m2 <0 and m3 >=0 and (m2+m3)<0:
                ws.cell(row,8).value = (m1+m2+m3)
                ws.cell(row,9).value = 0
                ws.cell(row,10).value = 0
            elif m1>=0 and m2 <0 and m3 >=0 and (m2+m3)>=0:
                ws.cell(row,10).value = (m2+m3)
                ws.cell(row,9).value = 0
            elif m1 <0 and m2 >=0 and m3 >=0 and (m1+m2)<0:
                ws.cell(row,10).value = (m1+m2+m3)
                ws.cell(row,9).value = 0
                ws.cell(row,8).value = 0
            elif m1 <0 and m2 >=0 and m3 >=0 and (m1+m2)>=0:
                ws.cell(row,9).value = (m1+m2)
                ws.cell(row,8).value = 0
                
            elif m1 <0 and m2 <0 and m3 >0 and (m1+m2+m3)>=0:
                ws.cell(row,10).value = (m1+m2+m3)
                ws.cell(row,9).value = 0
                ws.cell(row,8).value = 0
            elif m1 >0 and m2 <0 and m3 <0 and (m1+m2+m3)>=0:
                ws.cell(row,8).value = (m1+m2+m3)
                ws.cell(row,9).value = 0
                ws.cell(row,10).value = 0
            elif m1 <0 and m2 >0 and m3 <0 and (m1+m2+m3)>=0:
                ws.cell(row,9).value = (m1+m2+m3)
                ws.cell(row,8).value = 0
                ws.cell(row,10).value = 0

            if m4<0:
                ws.cell(row,11).value=0
            if m5<0:
                ws.cell(row,12).value=0

            ws.cell(row,13).value= (ws.cell(row,8).value+ws.cell(row,9).value+ws.cell(row,10).value+ws.cell(row,11).value+ws.cell(row,12).value)
            if ws.cell(row,13).value == 0:
                del_row.append(row)
        for row in sorted(del_row,reverse=True):
            ws.delete_rows(row)
        ws.insert_rows(2)
        ws.insert_cols(9)
        ws.insert_cols(11)
        ws.insert_cols(13)
        ws.insert_cols(15)
        ws.insert_cols(17)
        ws.insert_cols(19)
        for row in range(3,ws.max_row + 1 ):
            ws.cell(row,9).value = "=$G{}*H{}".format(row,row)
            ws.cell(row,11).value = "=$G{}*J{}".format(row,row)
            ws.cell(row,13).value = "=$G{}*L{}".format(row,row)
            ws.cell(row,15).value = "=$G{}*N{}".format(row,row)
            ws.cell(row,17).value = "=$G{}*P{}".format(row,row)
            ws.cell(row,19).value = '=SUM(Q{},O{},M{},K{},I{})'.format(row,row,row,row,row)
        ws['E2']='Grand Total'
        ws['H1']='{} ATS Qty'.format(mon)
        ws['I1']='{} ATS $'.format(mon)
        ws['J1']='{} ATS Qty'.format(next_mon)
        ws['K1']='{} ATS $'.format(next_mon)
        ws['L1']='{} ATS Qty'.format(two_mon)
        ws['M1']='{} ATS $'.format(two_mon)
        ws['N1']='{} ATS Qty'.format(three_mon)
        ws['O1']='{} ATS $'.format(three_mon)
        ws['P1']='{} ATS Qty'.format(four_mon)
        ws['Q1']='{} ATS $'.format(four_mon)
        ws['S1']='Total ATS $'
        ws['H2']='=SUM(H3:H{})'.format(ws.max_row)
        ws['I2']='=SUM(I3:I{})'.format(ws.max_row)
        ws['J2']='=SUM(J3:J{})'.format(ws.max_row)
        ws['K2']='=SUM(K3:K{})'.format(ws.max_row)
        ws['L2']='=SUM(L3:L{})'.format(ws.max_row)
        ws['M2']='=SUM(M3:M{})'.format(ws.max_row)
        ws['N2']='=SUM(N3:N{})'.format(ws.max_row)
        ws['O2']='=SUM(O3:O{})'.format(ws.max_row)
        ws['P2']='=SUM(P3:P{})'.format(ws.max_row)
        ws['Q2']='=SUM(Q3:Q{})'.format(ws.max_row)
        ws['R2']='=SUM(R3:R{})'.format(ws.max_row)
        ws['S2']='=SUM(S3:S{})'.format(ws.max_row)
        ws['T2']='=SUM(T3:T{})'.format(ws.max_row)
        ws['U2']='=SUM(U3:U{})'.format(ws.max_row)
        
        for cell in ws['I']:
                cell.font = copy.copy(ws['H3'].font)
                cell.fill = copy.copy(ws['H3'].fill)
                cell.border = copy.copy(ws['H3'].border)
                cell.alignment = copy.copy(ws['H3'].alignment)
                cell.number_format = '"$"#,##0'
        for cell in ws['K']:
                cell.font = copy.copy(ws['H3'].font)
                cell.fill = copy.copy(ws['H3'].fill)
                cell.border = copy.copy(ws['H3'].border)
                cell.alignment = copy.copy(ws['H3'].alignment)
                cell.number_format = '"$"#,##0'
        for cell in ws['M']:
                cell.font = copy.copy(ws['H3'].font)
                cell.fill = copy.copy(ws['H3'].fill)
                cell.border = copy.copy(ws['H3'].border)
                cell.alignment = copy.copy(ws['H3'].alignment)
                cell.number_format = '"$"#,##0'
        for cell in ws['O']:
                cell.font = copy.copy(ws['H3'].font)
                cell.fill = copy.copy(ws['H3'].fill)
                cell.border = copy.copy(ws['H3'].border)
                cell.alignment = copy.copy(ws['H3'].alignment)
                cell.number_format = '"$"#,##0'
        for cell in ws['Q']:
                cell.font = copy.copy(ws['H3'].font)
                cell.fill = copy.copy(ws['H3'].fill)
                cell.border = copy.copy(ws['H3'].border)
                cell.alignment = copy.copy(ws['H3'].alignment)
                cell.number_format = '"$"#,##0'
        for cell in ws['S']:
                cell.font = copy.copy(ws['H3'].font)
                cell.fill = copy.copy(ws['H3'].fill)
                cell.border = copy.copy(ws['H3'].border)
                cell.alignment = copy.copy(ws['H3'].alignment)
                cell.number_format = '"$"#,##0'
        
        for rows in ws.iter_rows(max_row=2):
            for cell in rows:
                cell.font = copy.copy(ws['A1'].font)
                cell.fill = copy.copy(ws['A1'].fill)
                cell.border = copy.copy(ws['A1'].border)
                cell.alignment = copy.copy(ws['A1'].alignment)

**Manule Work**\
Write down the file path for Python and run.\
For ATS file from SAP, delete the first 3 rows manually. No needs for Monitor, but LFD. CHECK FILES first. SAP format keeps changing.

In [18]:
# ATS files without first 3 rows !!!

ats="C:\\Users\\zhuoyan.bai\\Documents\\ATS\\wk16\\LFD_ATS_04142025WK16.xlsx"

gscm(ats,
      "C:\\Users\\zhuoyan.bai\\Documents\\ATS\\wk16\\arr.xlsx","PSI Arrivals")
gscm(ats,
      "C:\\Users\\zhuoyan.bai\\Documents\\ATS\\wk16\\ap1.xlsx","AP1")
sap(ats,
     "C:\\Users\\zhuoyan.bai\\Documents\\ATS\\wk16\\ats.xlsx","On-Hand")
sap(ats,"C:\\Users\\zhuoyan.bai\\Documents\\ATS\\wk16\\unship.xlsx","Open SO")

# OLD Format
# with pd.ExcelWriter(ats, mode='a', engine='openpyxl', if_sheet_exists="overlay") as writer:
#     workbook = writer.book
#     del workbook['Summary']
#     ws = workbook.copy_worksheet(workbook['Working'])
#     ws.title = "Summary"
#     ws.sheet_properties.tabColor = '00FFFF00'
#     workbook.move_sheet('Summary',-7)


After All, let's output the ATS Excel file and ready for sending the email.

In [20]:

Sum(ats,today)
    